# Underwater obstacle detection Pipeline

In [ ]:
#importing the necessary packages
import numpy as np
import cv2
import imutils

In [ ]:
cap = cv2.VideoCapture('/path/to/video/')
while(cap.isOpened()):
    ret, image = cap.read() #getting our input image
    if ret==True:
        # resizing input image and making a copy of input image
        image = imutils.resize(image, width=1800) 
        output = image.copy()

        # Mask to detect the black line in image
        black_low = (0,0,0) #BGR values of black
        black_high = (90,80,60)
        mask_black = cv2.inRange(image, black_low, black_high)
        
        # Creating our Region of interest(ROI)
        l_left= [300,mask_black.shape[0]]
        t_left = [600,mask_black.shape[0]//2.5]
        l_right = [1500,mask_black.shape[0]]
        t_right = [1200,mask_black.shape[0]//2.5]
        vertices = [np.array([l_left,t_left,t_right,l_right],dtype=np.int32)]
        roi_mask = np.zeros_like(mask_black)
        cv2.fillPoly(roi_mask, vertices, 255)
        roi_image = cv2.bitwise_and(mask_black, roi_mask)
        
        # Shading the identified lane with green in original image
        output[roi_image == 255] = np.array([102, 255, 102]) #BGR value of green
        
        # Mask to detect the yellow obstacle in between
        light_yl = (100,90,70)
        dark_yl = (120,150,90)
        mask_yl = cv2.inRange(image, light_yl, dark_yl)
        
        # Applying gaussian blur to reduce noise and Canny Edge Detector to detect the obstacle
        # Parameter values got through trail and error
        gauss = cv2.GaussianBlur(mask_yl,(5,5),0)
        edged = cv2.Canny(gauss, 200,400)

        # Applying Probablistic Hough Transformation to find and draw the lines detected using Canny Edge detector
        min_threshold = 80
        lines = cv2.HoughLinesP(edged, cv2.HOUGH_PROBABILISTIC, np.pi/180, min_threshold, np.array([]),
                                minLineLength=40, maxLineGap=75)
        if lines is not None:
            for line in lines:
                #Drawing the lines on the original image 
                x1, y1, x2, y2 = line[0]
                pts = np.array([[x1, y1 ], [x2 , y2]], np.int32)
                cv2.polylines(output, [pts], True, (0,0,255),5)
        
        # Displaying the final output
        cv2.imshow('Output', output)

        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()